In [1]:
import numpy as np

In [ ]:
class Dense:
  def __init__(self, num_inputs, num_outputs):
    ni = num_inputs
    no = num_outputs
    self.w = np.sqrt(2/ni)*np.random.randn(ni, no)
    self.b = np.zeros(no)
    self.x = None
    self.dw = None
    self.db = None

  def forward(self, x):
    self.x = x
    out = np.dot(x, self.w) + self.b
    return out

  def backward(self, dL):
    dx = np.dot(dL, self.w.T)
    self.dw = np.dot(self.x.T, dL)
    self.db = np.sum(dL, axis=0)
    return dx

  def update(self, lr=0.001):
    self.w -= lr*self.dw
    self.b -= lr*self.db

In [ ]:
class Relu:
  def __init__(self):
    self.mask = None

  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0
    return out

  def backward(self, dL):
    dL[self.mask] = 0
    return dL

  def update(self, lr = 0.001):
    pass

In [ ]:
class Softmax:
  def __init__(self):
    pass

  def forward(self, x):
    if x.ndim == 1:
      c = np.max(x)
      exp_x = np.exp(x-c)
      sum_ = np.sum(exp_x)
    elif x.ndim == 2:
      c = np.max(x,axis=1).reshape(-1, 1)
      exp_x = np.exp(x-c)
      sum_ = np.sum(exp_x, axis=1).reshape(-1, 1)

    y = exp_x / sum_

    return y

  def backward(self, dL):
    pass

  def update(self, lr = 0.001):
    pass

In [ ]:
def cross_entropy(y, t):
    epsilon = 1e-7
    batch_size = y.shape[0]
    y = np.clip(y, epsilon, 1 - epsilon) 
    loss = -np.sum(t * np.log(y)) / batch_size
    return loss

In [ ]:
class SoftmaxWithLoss:
  def __init__(self):
    self.y = None
    self.t = None

  def forward(self, y, t):
    if y.ndim != t.ndim:
      self.t = np.eye(y.shape[1])[t]
    else:
      self.t =t

    self.y = y
    loss = cross_entropy(self.y, self.t)
    return loss

  def backward(self):
    batch_size = self.y.shape[0]
    dx = (self.y - self.t)/batch_size
    return dx

In [ ]:
class ANN:
  def __init__(self):

    self.Layers = []

    self.Layers.append(Dense(6, 12))
    self.Layers.append(Relu())
    self.Layers.append(Dense(12, 8))
    self.Layers.append(Relu())
    self.Layers.append(Dense(8, 6))
    self.Layers.append(Relu())
    self.Layers.append(Dense(6, 6))

    self.RevLayers = self.Layers.copy()
    self.RevLayers.reverse()

    self.Loss = SoftmaxWithLoss()

  def forward(self, x):
    for layer in self.Layers:
      x = layer.forward(x)
    return x

  def predict_max(self, x):
    x = self.forward(x)
    return np.argmax(x, axis=(x.ndim-1))

  def predict_min(self, x):
    x = self.forward(x)
    return np.argmin(x, axis=(x.ndim-1))

  def accuracy_max(self, x, t, batch_size = 10):
    b = batch_size
    N = x.shape[0]//b
    Nt=0
    for i in range(N):
      xd = x[i*b:(i+1)*b]
      td = t[i*b:(i+1)*b]
      yd = self.predict_max(xd)

      Nt += np.sum(yd==td)
    acc = Nt/(N*b)
    return acc

  def accuracy_min(self, x, t, batch_size = 10):
    b = batch_size
    N = x.shape[0]//b
    Nt=0
    for i in range(N):
      xd = x[i*b:(i+1)*b]
      td = t[i*b:(i+1)*b]
      yd = self.predict_min(xd)

      Nt += np.sum(yd==td)
    acc = Nt/(N*b)
    return acc

  def loss(self, x, t):
    y = self.forward(x)
    L = self.Loss.forward(y,t)
    return L

  def backward(self):
    dL = self.Loss.backward()
    for layer in self.RevLayers:
      dL = layer.backward(dL)
    return dL

  def update(self, lr=0.01):
    for layer in self.RevLayers:
      layer.update(lr)

  def fit(self, x, t, batch_size = 10, epoch=100, lr=0.01):
    b= batch_size
    N = x.shape[0]//b
    Mask = np.arange(x.shape[0])
    for e in range(epoch):
      np.random.shuffle(Mask)
      for i in range(N):
        xd = x[Mask[i*b:(i+1)*b]]
        td = t[Mask[i*b:(i+1)*b]]

        L = self.loss(xd,td)
        dL = self.backward()
        self.update(lr = lr)

    print(f"\r loss value = {L}", end="")


In [ ]:
# 학습 데이터 생성

np.random.seed(0) 

X = np.random.randint(100, size=(1000, 6)) / 100
Y_max = np.argmax(X, axis=1)

Y_min = np.argmin(X, axis=1)


In [ ]:
model_max = ANN()
model_max.fit(X, Y_max, batch_size=20, epoch = 1000, lr=0.01)
print("최대값 위치 정확도:", model_max.accuracy_max(X, Y_max))

model_min = ANN()
model_min.fit(X, Y_min, batch_size=20, epoch = 1000, lr=0.01)
print("최소값 위치 정확도:", model_min.accuracy_min(X, Y_min))

 loss value = 0.41363304874892315최대값 위치 정확도: 0.85
 loss value = 0.8358774322578675최소값 위치 정확도: 0.004


In [ ]:
X_test = np.array([[0.3, 0.5, 0.31, 0.55, 0.01, 0.9]])
Y_test_max = np.argmax(X_test)
Y_test_min = np.argmin(X_test)

print(Y_test_max)
print(Y_test_min)

5
4
